In [1]:
#Imports
import time
import sys
import os
sys.path.append('../../')

from pyspark.ml import Pipeline, PipelineModel
from sparknlp.annotator import *
from sparknlp.base import DocumentAssembler, Finisher


In [2]:
#Load the input data to be annotated
data = spark. \
        read. \
        parquet("file:///" + os.getcwd() + "/../../../src/test/resources/sentiment.parquet"). \
        limit(1000)
data.cache()
data.count()
data.show()

+------+---------+--------------------+
|itemid|sentiment|                text|
+------+---------+--------------------+
|     1|        0|                 ...|
|     2|        0|                 ...|
|     3|        1|              omg...|
|     4|        0|          .. Omga...|
|     5|        0|         i think ...|
|     6|        0|         or i jus...|
|     7|        1|       Juuuuuuuuu...|
|     8|        0|       Sunny Agai...|
|     9|        1|      handed in m...|
|    10|        1|      hmmmm.... i...|
|    11|        0|      I must thin...|
|    12|        1|      thanks to a...|
|    13|        0|      this weeken...|
|    14|        0|     jb isnt show...|
|    15|        0|     ok thats it ...|
|    16|        0|    &lt;-------- ...|
|    17|        0|    awhhe man.......|
|    18|        1|    Feeling stran...|
|    19|        0|    HUGE roll of ...|
|    20|        0|    I just cut my...|
+------+---------+--------------------+
only showing top 20 rows



In [3]:
### Define the dataframe
document_assembler = DocumentAssembler() \
            .setInputCol("text")
        
### Transform input to appropriate schema
#assembled = document_assembler.transform(data)

In [4]:
### Sentence detector
sentence_detector = SentenceDetector() \
    .setInputCols(["document"]) \
    .setOutputCol("sentence")
#sentence_data = sentence_detector.transform(checked)

In [5]:
### Tokenizer
tokenizer = Tokenizer() \
            .setInputCols(["sentence"]) \
            .setOutputCol("token")
#tokenized = tokenizer.transform(assembled)

In [6]:
normalizer = Normalizer() \
            .setInputCols(["token"]) \
            .setOutputCol("normal")

In [7]:
### Spell Checker
spell_checker = NorvigSweetingApproach() \
            .setInputCols(["normal"]) \
            .setOutputCol("spell")

#checked = spell_checker.fit(tokenized).transform(tokenized)

In [ ]:
sentiment_detector = ViveknSentimentApproach() \
    .setInputCols(["spell", "sentence"]) \
    .setOutputCol("sentiment") \
    .setPruneCorpus(0) \
    .setPositiveSource("file:///" + os.getcwd() + "/../../../src/test/resources/vivekn/positive") \
    .setNegativeSource("file:///" + os.getcwd() + "/../../../src/test/resources/vivekn/negative") \


In [ ]:
pos = PerceptronApproach() \
    .setInputCols(["sentence", "spell"]) \
    .setOutputCol("pos") \
    .setCorpus("file:///" + os.getcwd() + "/../../../src/test/resources/anc-pos-corpus-small/", delimiter="|")

In [ ]:
finisher = Finisher() \
    .setInputCols(["sentiment"]) \
    .setIncludeKeys(True)

In [8]:
training = spark.sparkContext.parallelize([["hello world"], ["this is some more text"], ["and here another sentence"]]).toDF().toDF("text")
training.show()

+--------------------+
|                text|
+--------------------+
|         hello world|
|this is some more...|
|and here another ...|
+--------------------+



In [10]:
pipeline = Pipeline(stages=[
    document_assembler,
    sentence_detector,
    tokenizer,
    normalizer,
    spell_checker
])

start = time.time()
sentiment_data = pipeline.fit(training).transform(data)
sentiment_data.show()
end = time.time()
print("Time elapsed pipeline process: " + str(end - start))

+------+---------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|itemid|sentiment|                text|            document|            sentence|               token|              normal|               spell|
+------+---------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|     1|        0|                 ...|[[document, 0, 60...|[[document, 21, 4...|[[token, 21, 22, ...|[[token, 21, 22, ...|[[token, 21, 22, ...|
|     2|        0|                 ...|[[document, 0, 50...|[[document, 19, 4...|[[token, 19, 19, ...|[[token, 19, 19, ...|[[token, 19, 19, ...|
|     3|        1|              omg...|[[document, 0, 36...|[[document, 14, 3...|[[token, 14, 16, ...|[[token, 14, 16, ...|[[token, 14, 16, ...|
|     4|        0|          .. Omga...|[[document, 0, 13...|[[document, 10, 1...|[[token, 10, 10, ...|[[token, 13, 18, ...|[[token

In [ ]:
for r in sentiment_data.take(5):
    print(r)

In [ ]:
start = time.time()
pipeline.write().overwrite().save("./ps")
pipeline.fit(data).write().overwrite().save("./ms")
end = time.time()
print("Time elapsed in write pipelines: " + str(end - start))

In [ ]:
start = time.time()
p = Pipeline.read().load("./ps")
pm = PipelineModel.read().load("./ms")
end = time.time()
print("Time elapsed in read pipelines: " + str(end - start))

In [ ]:
start = time.time()
pm.transform(data).where("finished_sentiment not like '%negative%'").show()
print(pm.transform(data).count())
end = time.time()
print("Time elapsed in using loaded pipelines: " + str(end - start))